In [16]:
from abc import ABCMeta

import qutip as qt

from math import *

import numpy as np

import scipy as sp
import scipy.integrate as spint
import scipy.linalg as splinalg
import scipy.optimize as spopt

import plotly.express as px

import matplotlib.pyplot as plt

Implemet simple ´qaoa´ solver for general Hamiltonian given as $Z$-polynomial

$$
\begin{equation}
    H = C + \sum_{i}c_{i}Z_{i} + \sum_{i,j}c_{i,j}Z_{i}Z_{j} + \sum_{i,j,k}c_{i,j,k}Z_{i}Z_{j}Z_{k} + \dots
\end{equation}
$$
\\
For that write global functions for all parts of `qaoa` $\rightarrow$ *trial state preperation*, *measurement*, computing the *expectation* .
Write custom, abstract `OPTIMIZER` class wich can be substituted by scipy optimizers like `cobyla` and then later by the `tdvp optimizer`. 

In [17]:
def trial_state(hamiltonian, betas, gammas, mixer , initial_state):
    pass

In [18]:
def expectation(circuit, function) -> float:
    pass

In [ ]:
class OPTIMIZER(metaclass=ABCMeta):
    def __init__():
        pass